In [51]:
import matlab.engine
import numpy as np

In [54]:
eng = matlab.engine.start_matlab()

def dimensionalise(x, **P):
    max_omega = P.pop('max_omega', 1)
    period = P.pop('period', 1)
    return np.hstack(((x[:,0]*period).reshape(-1,1), x[:,1:]*max_omega))

def batch_into_timeseries_struc(x_batch):
    x_ts = []
    for x in x_batch:
        time = x[:,0]
        data = x[:,1:] 
        matlab_time = matlab.double(time.tolist())
        matlab_data = matlab.double(data.tolist())
        x_ts += [eng.timeseries(matlab_data, matlab_time)];
    return eng.cat(1, *x_ts)

def simulink_results2fitness(results):
    return np.asarray(results).T[:,:,-1]

# Constructor
test = np.array([[0., 0., 0., 0.1],
                 [0.5, 0., 0., 0.1],
                 [1.0, 0., 0., 0.1]])

In [ ]:
P = 6.5293e+03
max_omega = 0.0096
x_test = dimensionalise(test, period=P, max_omega=max_omega)
x_test=x_test.reshape(1,*x_test.shape)
x_batch = np.tile(x_test, (30,1,1))

ipn = batch_into_timeseries_struc(x_batch)
res = eng.batch_sim('attitude3d', ipn)
fitness = simulink_results2fitness(res)